# DSCI 525: Milestone 1

Group 7:
- Alex Truong Hai Yen
- Peter Yang
- Joshua JiaJie Lim
- Rahul Kuriyedath

**Note to TA:** The running time and peak memory usage results vary for each rerun of the notebook. Moreover, the results are also computer-dependent. The notebook was run from start to end using Peter's PC (Windows 16Gb RAM)

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage

In [2]:
%load_ext rpy2.ipython

C:\Users\szyan\miniconda3\envs\525\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [3]:
%load_ext memory_profiler

## A) Downloading the data

In [4]:
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshareairline/"

In [5]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

In [6]:
%%time
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

Wall time: 29.1 s


In [7]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

Wall time: 15.2 s


## B) Combining data CSVs

In [8]:
import pandas as pd
files = glob.glob('figshareairline/*.csv')
files

['figshareairline\\ACCESS-CM2_daily_rainfall_NSW.csv',
 'figshareairline\\ACCESS-ESM1-5_daily_rainfall_NSW.csv',
 'figshareairline\\AWI-ESM-1-1-LR_daily_rainfall_NSW.csv',
 'figshareairline\\BCC-CSM2-MR_daily_rainfall_NSW.csv',
 'figshareairline\\BCC-ESM1_daily_rainfall_NSW.csv',
 'figshareairline\\CanESM5_daily_rainfall_NSW.csv',
 'figshareairline\\CMCC-CM2-HR4_daily_rainfall_NSW.csv',
 'figshareairline\\CMCC-CM2-SR5_daily_rainfall_NSW.csv',
 'figshareairline\\CMCC-ESM2_daily_rainfall_NSW.csv',
 'figshareairline\\EC-Earth3-Veg-LR_daily_rainfall_NSW.csv',
 'figshareairline\\FGOALS-f3-L_daily_rainfall_NSW.csv',
 'figshareairline\\FGOALS-g3_daily_rainfall_NSW.csv',
 'figshareairline\\GFDL-CM4_daily_rainfall_NSW.csv',
 'figshareairline\\GFDL-ESM4_daily_rainfall_NSW.csv',
 'figshareairline\\INM-CM4-8_daily_rainfall_NSW.csv',
 'figshareairline\\INM-CM5-0_daily_rainfall_NSW.csv',
 'figshareairline\\KIOST-ESM_daily_rainfall_NSW.csv',
 'figshareairline\\MIROC6_daily_rainfall_NSW.csv',
 'figsha

In [9]:
# For Mac OS:

# if 'figshareairline/combined_data.csv' in (files):
#     files.remove('figshareairline/combined_data.csv')
# if 'figshareairline/observed_daily_rainfall_SYD.csv' in (files):
#     files.remove('figshareairline/observed_daily_rainfall_SYD.csv')

In [10]:
# For Windows:

if 'figshareairline\\combined_data.csv' in (files):
    files.remove('figshareairline\\combined_data.csv')
if 'figshareairline\\observed_daily_rainfall_SYD.csv' in (files):
    files.remove('figshareairline\\observed_daily_rainfall_SYD.csv')

In [11]:
%%time
%memit

import pandas as pd # we do not use use_cols trick because we are loading all columns in the file
df = pd.concat((pd.read_csv(file)
                .assign(model=re.findall(r'[^\\/]+(?=_daily_rainfall_NSW)', file)[0])
                for file in files)
              )
df.to_csv("figshareairline/combined_data.csv", index = False)

peak memory: 136.30 MiB, increment: 0.28 MiB
Wall time: 6min 3s


In [12]:
%%sh
du -sh figshareairline/combined_data.csv

5.7G	figshareairline/combined_data.csv


In [13]:
%%time
df = pd.read_csv("figshareairline/combined_data.csv")

Wall time: 1min 1s


In [14]:
print(df.shape)

(62467843, 7)


In [15]:
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1,1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
2,1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
3,1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
4,1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2


**Comparison in run times and memory usages on different machines**

Machine | Peter (Windows 16Gb RAM) |Alex (Windows 8Gb RAM) |Joshua (macOS Catalina 8Gb RAM) |Rahul 
---|---|---|---|---
Download data - wall time| 29.1s | 12min 10s | 4min 10s
Unzip data - wall time| 15.2s| 22.5s | 21.4 s
Combine data - wall time| 6min 3s| 8 mins 16s | 7min 14s
Combine data - peak memory | 136.30 MiB| 93.20 MiB| 78.52 MiB
Read combined data - wall time | 1min 1s | 1min 50s | 1min 29s

**Comment:**

- Peter's machine can run the tasks faster than Alex's and Joshua's given the bigger RAM
- Given the same RAM size for Alex's and Joshua's machine, MacOS seems to process and tast faster than Windows.
- Alex's machine takes the longest to download the data.
- Rahul was not able to run this notebook given he just moved to Vancouver and did not have the suitable laptop adapter to work on this milestone.

## C) Load combined CSV to memory and perform a simple EDA in Python

### 1. How many records in each type of model?
#### 1.1. Classical way - load all to memory (for comparison later)

In [16]:
%%time
%%memit
df = pd.read_csv("figshareairline/combined_data.csv")
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
CMCC-CM2-SR5        3541230
CMCC-CM2-HR4        3541230
NorESM2-MM          3541230
CMCC-ESM2           3541230
TaiESM1             3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-ESM4           3219300
GFDL-CM4            3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM4-8           1609650
INM-CM5-0           1609650
FGOALS-g3           1287720
KIOST-ESM           1287720
MPI-ESM1-2-LR        966420
AWI-ESM-1-1-LR       966420
NESM3                966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
CanESM5              551880
BCC-ESM1             551880
Name: model, dtype: int64
peak memory: 11657.07 MiB, increment: 6534.23 MiB
Wall time: 58.8 s


#### 1.2. Loading in chunk 

In [17]:
%%time
%%memit
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("figshareairline/combined_data.csv", chunksize=10_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
dtype: int32
peak memory: 9624.98 MiB, increment: 2161.25 MiB
Wall time: 55.3 s


**Comment:** (Results are computer-dependent)

We cut the peak memory savings from 11657.07 MiB to 9624.98 MiB by loading the data in chunks instead of loading all data at once. There is savings in running time from 58.8 to 55.3s.

### 2. What is the average rain fall (mm/day) for the respective models
#### 2.1. Classical way - load all to memory (for comparison later)

In [18]:
%%time
%%memit
df = pd.read_csv("figshareairline/combined_data.csv")
print(df.groupby(by = ['model']).mean()['rain (mm/day)'])

model
ACCESS-CM2          1.787025
ACCESS-ESM1-5       2.217501
AWI-ESM-1-1-LR      2.026071
BCC-CSM2-MR         1.951832
BCC-ESM1            1.811032
CMCC-CM2-HR4        2.279350
CMCC-CM2-SR5        2.383389
CMCC-ESM2           2.266125
CanESM5             1.894328
EC-Earth3-Veg-LR    1.516258
FGOALS-f3-L         1.627373
FGOALS-g3           2.156419
GFDL-CM4            1.414485
GFDL-ESM4                NaN
INM-CM4-8           2.811463
INM-CM5-0           2.669012
KIOST-ESM           1.102353
MIROC6              2.301662
MPI-ESM-1-2-HAM     1.610720
MPI-ESM1-2-HR       0.995569
MPI-ESM1-2-LR       1.074308
MRI-ESM2-0          1.368030
NESM3               1.621936
NorESM2-LM          2.230799
NorESM2-MM          2.232966
SAM0-UNICON         2.169676
TaiESM1             2.224576
Name: rain (mm/day), dtype: float64
peak memory: 11833.14 MiB, increment: 4152.84 MiB
Wall time: 1min 2s


#### 2.2 Only load the columns that we are interested in

In [19]:
%%time
%%memit
use_cols = ['model','rain (mm/day)']
df = pd.read_csv("figshareairline/combined_data.csv", usecols = use_cols)
print(df.groupby(by = ['model']).mean()['rain (mm/day)'])

model
ACCESS-CM2          1.787025
ACCESS-ESM1-5       2.217501
AWI-ESM-1-1-LR      2.026071
BCC-CSM2-MR         1.951832
BCC-ESM1            1.811032
CMCC-CM2-HR4        2.279350
CMCC-CM2-SR5        2.383389
CMCC-ESM2           2.266125
CanESM5             1.894328
EC-Earth3-Veg-LR    1.516258
FGOALS-f3-L         1.627373
FGOALS-g3           2.156419
GFDL-CM4            1.414485
GFDL-ESM4                NaN
INM-CM4-8           2.811463
INM-CM5-0           2.669012
KIOST-ESM           1.102353
MIROC6              2.301662
MPI-ESM-1-2-HAM     1.610720
MPI-ESM1-2-HR       0.995569
MPI-ESM1-2-LR       1.074308
MRI-ESM2-0          1.368030
NESM3               1.621936
NorESM2-LM          2.230799
NorESM2-MM          2.232966
SAM0-UNICON         2.169676
TaiESM1             2.224576
Name: rain (mm/day), dtype: float64
peak memory: 7066.39 MiB, increment: 1910.34 MiB
Wall time: 35.8 s


#### 2.3 Using dask

In [20]:
%%time
%%memit
import dask.dataframe as dd

ddf = dd.read_csv('figshareairline/combined_data.csv', )
print(ddf.groupby(by = ['model']).mean()['rain (mm/day)'].compute())

model
ACCESS-CM2          1.787025
ACCESS-ESM1-5       2.217501
AWI-ESM-1-1-LR      2.026071
BCC-CSM2-MR         1.951832
BCC-ESM1            1.811032
CanESM5             1.894328
CMCC-CM2-HR4        2.279350
CMCC-CM2-SR5        2.383389
CMCC-ESM2           2.266125
EC-Earth3-Veg-LR    1.516258
FGOALS-f3-L         1.627373
FGOALS-g3           2.156419
GFDL-CM4            1.414485
GFDL-ESM4                NaN
INM-CM4-8           2.811463
INM-CM5-0           2.669012
KIOST-ESM           1.102353
MIROC6              2.301662
MPI-ESM-1-2-HAM     1.610720
MPI-ESM1-2-HR       0.995569
MPI-ESM1-2-LR       1.074308
MRI-ESM2-0          1.368030
NESM3               1.621936
NorESM2-LM          2.230799
NorESM2-MM          2.232966
SAM0-UNICON         2.169676
TaiESM1             2.224576
Name: rain (mm/day), dtype: float64
peak memory: 3470.72 MiB, increment: 2393.04 MiB
Wall time: 36.8 s


**Comment:** (Results are computer dependent)
- By loading only the columns of interest instead of all the columns in the dataframe, we can save peak memory usage from 11833.14 in 2.1 to  7066.39 MiB in 2.2. Running time is significantly reduced from 1min 2s to 35.8s.

- With dask where chunking and parallel execution are applied automatically for us, we can process the data faster than traditional Pandas method at 36.8 s. It also takes up the least amount of memory during peak usage at 3470.72 MiB, that is only around 50% of what the traditional Pandas method consumes.

## D) Transfer dataframe from Python to R and Perform a simple R EDA

### 1. Comparing the different file formats created in Python in terms of file size

(Results are computer dependent)

In [21]:
# Attribution: adopt code of Gittu in DSCI 525 Lecture 2 with modification

In [22]:
%%time
%%memit
# convert csv to arrow table format
import pyarrow.dataset as ds
dataset = ds.dataset("figshareairline/combined_data.csv", format="csv")
table = dataset.to_table()

peak memory: 4980.05 MiB, increment: 3883.42 MiB
Wall time: 22.9 s


In [23]:
%%time
%%memit
# writing arrow table to feather format
import pyarrow.feather as feather
feather.write_feather(table, 'figshareairline/combined_data.feather')

peak memory: 4998.50 MiB, increment: 21.58 MiB
Wall time: 2.47 s


In [24]:
%%time
%%memit
# writing arrow table to feather format
import pyarrow.parquet as pq
pq.write_to_dataset(table, 'figshareairline/combined_data.parquet')

peak memory: 5184.34 MiB, increment: 185.82 MiB
Wall time: 10.4 s


In [25]:
%%sh
# seeing the size of the csv data
du -sh figshareairline/combined_data.csv
# seeing the size of the feather data
du -sh figshareairline/combined_data.feather
# seeing the size of the parquet data
du -sh figshareairline/combined_data.parquet

5.7G	figshareairline/combined_data.csv
1.1G	figshareairline/combined_data.feather
542M	figshareairline/combined_data.parquet


### 2. Rationale for why we chose **parquet** file format to transfer dataframe from python to R:

- The traditional panda exchange would be inefficient in running time for both loading and process the data during EDA because with it has to load the entire data in memory. Moreover, its storage in csv file is the largest

- Arrow exchange helps reduce the running time as compared to traditional panda exchange thanks to its columnar memory format and language independence or no serialization features. Instead of having to load or search through the entire data, it can efficiently search by column. However, arrow format only supports some operations for data wrangling, this could be restricting on EDA.

- Hence, we preferred other columnar memory file formats like .feather or .pacquet.  because it has all the advantage of columnar format like arrow and it allows more flexible operations in data wrangling. 

- Between .feather and .pacquet, we chose .pacquet given it has a more established history and we think its performance would be more stable or less bugging than .feather, which is actively developed. Moreover, .pacquet file is highly portable because its file size is the smallest for the same amount of data.


*Reference: [What is Apache Parquet and why you should use it!](https://www.upsolver.com/blog/apache-parquet-why-use)*

### 3. Transfer data file to R using pacquet and conduct a simple R EDA

In [26]:
filepath = glob.glob('figshareairline/combined_data.parquet/*.parquet')[0]
filepath

'figshareairline/combined_data.parquet\\460369ff572d43b286e6ade5bafd370c.parquet'

In [27]:
%%time
%%R -i filepath

# Looading the data from parquet
library(arrow)
r_table <- arrow::read_parquet(filepath)

# conduct simple EDA

## EDA 1
library(dplyr)
start_time <- Sys.time()

result_eda1 <- r_table %>%
    group_by(model) %>%
    summarise(no_of_records = n()) 

end_time <- Sys.time()

print(result_eda1)
print(end_time - start_time)

## EDA 2
start_time <- Sys.time()

result_eda2 <- r_table %>%
    group_by(model) %>%
    summarise(average_rainfall = mean(`rain (mm/day)`)) 

end_time <- Sys.time()

print(result_eda2)
print(end_time - start_time)

R[write to console]: 
Attaching package: 'arrow'


R[write to console]: The following object is masked from 'package:utils':

    timestamp


R[write to console]: 
Attaching package: 'dplyr'


R[write to console]: The following objects are masked from 'package:stats':

    filter, lag


R[write to console]: The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




`summarise()` ungrouping output (override with `.groups` argument)
# A tibble: 27 x 2
   model            no_of_records
   <chr>                    <int>
 1 ACCESS-CM2             1932840
 2 ACCESS-ESM1-5          1610700
 3 AWI-ESM-1-1-LR          966420
 4 BCC-CSM2-MR            3035340
 5 BCC-ESM1                551880
 6 CanESM5                 551880
 7 CMCC-CM2-HR4           3541230
 8 CMCC-CM2-SR5           3541230
 9 CMCC-ESM2              3541230
10 EC-Earth3-Veg-LR       3037320
# ... with 17 more rows
Time difference of 1.699997 secs
`summarise()` ungrouping output (override with `.groups` argument)
# A tibble: 27 x 2
   model            average_rainfall
   <chr>                       <dbl>
 1 ACCESS-CM2                   1.79
 2 ACCESS-ESM1-5                2.22
 3 AWI-ESM-1-1-LR               2.03
 4 BCC-CSM2-MR                  1.95
 5 BCC-ESM1                     1.81
 6 CanESM5                      1.89
 7 CMCC-CM2-HR4                 2.28
 8 CMCC-CM2-SR5               

**Comment:** 

Besides 1.7s and 2.4s for the two respective EDA, the remaining wall time was spent to read in the .parquet file (12.6 - 1.7 - 2.4 = 8.5 s). Given its size, this is a decent performance.

## Closing comments

* Given the size of this data is quite large, the time it took us to run the code is quite long. We overcome this by apply more efficient loading method as well as converted the data to more advanced format like .parquet
* There are different file syntax between Windows and macOS that we have managed to debug successfully.
* The `rpy2` is another obstacle given two of our members could not run this extension in Jupyter Notebooks despite many debugging attempts. In the end, we decide to test run the R code in RStudios first and then paste it in the Jupyter Notebooks and ask the other member with the most robust machine to run it in Jupyter Notebooks. 